In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

In [2]:
col_list = [i for i in range(1, 20)]
data = pd.read_csv('all_data.csv', usecols = col_list)
data.drop(columns = ['label'], inplace = True)
data.head()

,text,Label_by_zihan,Label_by_ben,Label_by_nicole,average,Qwen 1.5 Chat (0.5B),Qwen 1.5 Chat (1.8B),Qwen 1.5 Chat (4B),Qwen 1.5 Chat (7B),Qwen 1.5 Chat (14B),Qwen 1.5 Chat (72B),LLaMA-2 Chat (70B),GPT-4o,GPT-3.5 Turbo,GPT-4 Turbo,GPT-4o W/O Reasoning,Sentiment Roberta,Custmized Bert
0,what do these '1/2 naked pics' have to do with...,3.0,3.0,4.0,3.333333,1.0,3.0,1.0,6.0,3.0,3.0,3.0,3.0,3.0,5.0,2.0,3.000812,3.385936
1,OH: “I had a blue penis while I was this” [pla...,5.0,5.0,6.0,5.333333,7.0,7.0,0.0,0.0,3.0,3.0,6.0,5.0,5.0,6.0,1.0,5.122762,6.015794
2,"That's coming, but I think the victims are goi...",5.0,6.0,5.0,5.333333,7.0,5.0,2.0,5.0,3.0,4.0,3.0,2.0,2.0,3.0,3.0,4.173433,4.166149
3,I think I may be finally in with the in crowd ...,7.0,9.0,7.0,7.666667,7.0,7.0,1.0,8.0,6.0,6.0,8.0,8.0,8.0,8.0,8.0,6.907494,7.103713
4,"Wow,first Hugo Chavez and now Fidel Castro. Da...",4.0,2.0,3.0,3.000000,7.0,7.0,2.0,2.0,3.0,4.0,3.0,3.0,2.0,2.0,2.0,4.960894,3.268210


## Hypothesis Testing

- Take most capable LLM model and compare it against conventional 
    - Ex. Gpt or Llama or Qwen against conventional 
    - Rank LLM's by the mean absolute error across the model data (`test_stats`)
- Use two-sample hypothesis test
- Data is shifted from the scoring to absolute difference (distance) from average human rating (`|model_score - average_human_score|`)

In [12]:
test_model_ind = np.where(test_stats == min(test_stats[:-2]))[0][0]
test_model = models[test_model_ind]
test_model

'Qwen 1.5 Chat (72B)'

We want to know if conventional models or LLMs perform better. Our data will be the absolute differences (AD) between model rating and human rating 

- $X \sim N(\mu_x, \sigma_x ^2)$ and $Y \sim N(\mu_y, \sigma_y ^2)$ are both the absolute difference between model X and model Y scoring and the average human score


| Anatomy of the hypothesis test |  Answer  |
|:------------------------------:|:--------:|
| Assumption                     | $$X \sim N(\mu_x, \sigma_x ^2)$$ and $$Y \sim N(\mu_y, \sigma_y ^2)$$ |
| Population parameter           | $$\theta = \mu_x - \mu_y$$ |
| Sample statistic               | $$\hat\theta = \overline{X} - \overline{Y}$$ |
| Test statistic                 | $$T = \displaystyle\frac{\hat\theta - \theta}{\text{SE}} \sim t_{k}$$ |
| Null hypothesis                | $$H_0: \theta = 0$$ |
| Alternate hypothesis           | $$H_a: \theta > 0$$ |
| Rejection region shape         | $$(x_\alpha, \infty)$$ |

- We will use $\alpha = 0.05$

In [13]:
best_llm = abs(data[test_model] - data['average'])
roberta = abs(data['Sentiment Roberta'] - data['average'])
bert = abs(data['Custmized Bert'] - data['average'])

berta_stat, berta_p = stats.ttest_ind(best_llm, roberta, alternative = 'greater')

# 'greater’: the mean of the distribution underlying the first sample is greater than the 
# mean of the distribution underlying the second sample; the llm absolute difference is greater than the
# roberta absolute difference 

/Users/nicole/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [14]:
bert_stat, bert_p = stats.ttest_ind(best_llm, bert, alternative = 'greater')

# 'greater’: the mean of the distribution underlying the first sample is greater than the 
# mean of the distribution underlying the second sample; the llm absolute difference is greater than the
# bert absolute difference 

In [33]:
def decide(alpha, p_value):
    if p_value < alpha:
        print(f'alpha = {alpha}, p_value = {p_value}')
        print('Reject null: LLMs have a larger mean -> LLMs have a larger error and a worse performance ')
    else:
        print(f'alpha = {alpha}, p_value = {p_value}')
        print('Fail to reject null: LLMs do not appear have a larger mean -> LLMs and conventional models have similar perfomances')
    
alpha = 0.05
decide(alpha, berta_p)

alpha = 0.05, p_value = 2.6596145689960566e-11
Reject null: LLMs have a larger mean -> LLMs have a larger error and a worse performance 


In [20]:
llm_response = data.iloc[:, 5:-2]
llm_response

,Qwen 1.5 Chat (0.5B),Qwen 1.5 Chat (1.8B),Qwen 1.5 Chat (4B),Qwen 1.5 Chat (7B),Qwen 1.5 Chat (14B),Qwen 1.5 Chat (72B),LLaMA-2 Chat (70B),GPT-4o,GPT-3.5 Turbo,GPT-4 Turbo,GPT-4o W/O Reasoning
0,1.0,3.0,1.0,6.0,3.0,3.0,3.0,3.0,3.0,5.0,2.0
1,7.0,7.0,0.0,0.0,3.0,3.0,6.0,5.0,5.0,6.0,1.0
2,7.0,5.0,2.0,5.0,3.0,4.0,3.0,2.0,2.0,3.0,3.0
3,7.0,7.0,1.0,8.0,6.0,6.0,8.0,8.0,8.0,8.0,8.0
4,7.0,7.0,2.0,2.0,3.0,4.0,3.0,3.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
245,0.0,5.0,6.0,1.0,3.0,5.0,3.0,3.0,2.0,4.0,1.0
246,7.0,7.0,1.0,7.0,6.0,8.0,8.0,8.0,8.0,8.0,8.0
247,5.0,1.0,6.0,6.0,4.0,5.0,3.0,3.0,3.0,3.0,2.0
248,9.0,10.0,0.0,9.0,7.0,5.0,8.0,8.0,9.0,8.0,8.0


In [21]:
for col in llm_response.columns:
    llm_response[col] = abs(llm_response[col] - data['average'])
perf = llm_response.mean(axis = 0).sort_values(ascending = True)
perf

Qwen 1.5 Chat (72B)     1.058667
GPT-4 Turbo             1.240000
Qwen 1.5 Chat (14B)     1.280000
LLaMA-2 Chat (70B)      1.369333
GPT-4o                  1.442667
GPT-3.5 Turbo           1.569333
GPT-4o W/O Reasoning    1.626667
Qwen 1.5 Chat (7B)      2.006667
Qwen 1.5 Chat (1.8B)    2.688000
Qwen 1.5 Chat (4B)      3.160000
Qwen 1.5 Chat (0.5B)    3.568000
dtype: float64

In [16]:
list(perf.keys())

['Qwen 1.5 Chat (72B)',
 'GPT-4 Turbo',
 'Qwen 1.5 Chat (14B)',
 'LLaMA-2 Chat (70B)',
 'GPT-4o',
 'GPT-3.5 Turbo',
 'GPT-4o W/O Reasoning',
 'Qwen 1.5 Chat (7B)',
 'Qwen 1.5 Chat (1.8B)',
 'Qwen 1.5 Chat (4B)',
 'Qwen 1.5 Chat (0.5B)']

In [24]:
alpha = 0.05
def decide_LLM(alpha, p_value):
    if p_value < alpha:
        print(f'alpha = {alpha}, p_value = {p_value}')
        print('Reject null: Model 1 is significantly better than Model 2')
    else:
        print(f'alpha = {alpha}, p_value = {p_value}')
        print('Fail to reject null: Model 1 is not significantly better than Model 2')

In [28]:
# Qwen 1.5 Chat (72B) vs GPT-4 Turbo (between different LLMs)
stat1, p_value1 = stats.ttest_ind(llm_response['Qwen 1.5 Chat (72B)'], llm_response['GPT-4 Turbo'], alternative = 'less')
decide_LLM(alpha, p_value1)

alpha = 0.05, p_value = 0.010857275426296862
Reject null: Model 1 is significantly better than Model 2


In [29]:
# GPT-4 Turbo vs LLaMA-2 Chat (70B) (between different LLMs)
stat2, p_value2 = stats.ttest_ind(llm_response['GPT-4 Turbo'], llm_response['LLaMA-2 Chat (70B)'], alternative = 'less')
decide_LLM(alpha, p_value2)

alpha = 0.05, p_value = 0.06650384176944449
Fail to reject null: Model 1 is not significantly better than Model 2


In [30]:
# GPT-4 Turbo vs GPT-4o (GPT4o is the newest model, but GPT-4 Turbo shows better performance in the test set. Is that significant?)
stat2, p_value2 = stats.ttest_ind(llm_response['GPT-4 Turbo'], llm_response['GPT-4o'], alternative = 'less')
decide_LLM(alpha, p_value2)

alpha = 0.05, p_value = 0.007313356292272543
Reject null: Model 1 is significantly better than Model 2


In [31]:
# Is Qwen 1.5 Chat (72B) significantly better than its smaller variant Qwen 1.5 Chat (14B)?
stat2, p_value2 = stats.ttest_ind(llm_response['Qwen 1.5 Chat (72B)'], llm_response['Qwen 1.5 Chat (14B)'], alternative = 'less')
decide_LLM(alpha, p_value2)

alpha = 0.05, p_value = 0.005031223723426022
Reject null: Model 1 is significantly better than Model 2
